# 6. Merging and Reshaping Data

In this chapter, we continue with some of the ways to manipulate data using the tidyverse packages. In particular, we will look at reshaping and merging data frames in order to get the data in the format we want. When reshaping data, we can convert our data between wide form (more columns, fewer rows) and long form (fewer columns, more rows). For example, we can use these data pivots to put our data into a tidy form. When merging data, we are combining information from multiple data frames into a single data frame. The key idea when merging data is to think about what the common information is between the data frames and which values we want to keep. 

For this chapter, we will use three data sets. The first data set is `covidcases`, which contains the weekly case and death counts by county in the United States for 2020, the second data set is `mobility`, which contains daily mobility estimates by state in 2020, and lastly we have `lockdowndates`, which contains the start and end dates for statewide stay at home orders. Take a look at the first few rows of each data frame below and read the documentation for the column descriptions.

In [1]:
library(tidyverse)
library(RforHDSdata)
library(lubridate)
data(covidcases)
data(lockdowndates)
data(mobility)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
head(covidcases)

state,county,week,weekly_cases,weekly_deaths
<chr>,<chr>,<dbl>,<int>,<int>
Alabama,Autauga,12,3,0
Alabama,Autauga,13,3,0
Alabama,Autauga,14,2,1
Alabama,Autauga,15,11,1
Alabama,Autauga,16,5,1
Alabama,Autauga,17,8,2


In [3]:
head(mobility)

state,date,samples,m50,m50_index
<chr>,<chr>,<int>,<dbl>,<dbl>
Alabama,2020-03-01,267652,10.87194,76.92647
Alabama,2020-03-02,287264,14.34513,98.57353
Alabama,2020-03-03,292018,14.24460,98.25000
Alabama,2020-03-04,298704,13.08301,89.69118
Alabama,2020-03-05,288218,14.81503,102.38235
Alabama,2020-03-06,282982,17.94399,126.22059


In [4]:
head(lockdowndates)

State,Lockdown_Start,Lockdown_End
<chr>,<chr>,<chr>
Alabama,2020-04-04,2020-04-30
Alaska,2020-03-28,2020-04-24
Arizona,2020-03-31,2020-05-15
Arkansas,None,None
California,2020-03-19,2020-08-28
Colorado,2020-03-26,2020-04-26


 For each of these data frames, there is a date column. Right now the class of the date column in the lockdown and mobility data is a character value. We can use the `as.Date()` function to tell R to treat these as dates. We need to specify the date format as an argument to this function. In our format `%Y-%M-%D` the `%Y` stands for the full four-digit year, `%M` is a two-digit month (e.g. January is coded "01" vs "1"), and `%D` stands for the two-digit day (e.g. the third day is coded "03" vs "3"). 

In [5]:
mobility$date <- as.Date(mobility$date, formula="%Y-%M-%D")
lockdowndates$Lockdown_Start <- as.Date(lockdowndates$Lockdown_Start, formula="%Y-%M-%D")
lockdowndates$Lockdown_End <- as.Date(lockdowndates$Lockdown_End, formula="%Y-%M-%D")

By coding these columns as dates, we can access information such as the day, month, year, or week. These functions are all available in the `lubridate` package, which allows us to manipulate dates. 

In [6]:
month(mobility$date[1])
week(mobility$date[1])

[1] 3

[1] 9

Last, we add a date column to the covid cases data. In this case, we need to use the week number to find the date. Luckily, we can add days, months, weeks, or years to dates in the `lubridate` package. We add the week number recorded to the first day of 2020. 

In [7]:
covidcases$date <- as.Date("2020-01-01")+weeks(covidcases$week)
head(covidcases)

state,county,week,weekly_cases,weekly_deaths,date
<chr>,<chr>,<dbl>,<int>,<int>,<date>
Alabama,Autauga,12,3,0,2020-03-25
Alabama,Autauga,13,3,0,2020-04-01
Alabama,Autauga,14,2,1,2020-04-08
Alabama,Autauga,15,11,1,2020-04-15
Alabama,Autauga,16,5,1,2020-04-22
Alabama,Autauga,17,8,2,2020-04-29


## Tidy Data 

The tidyverse is designed around interacting with **tidy data** with the premise that a tidy format we can streamline our analysis. Data is considered **tidy** if

1.  Each variable is associated with a single column.
2.  Each observation is associated with a single row.
3.  Each value has its own cell.

Take a look at the data below which stores information about the maternal mortality rate for five countries over time (source: https://ourworldindata.org/maternal-mortality). This data is *not* tidy because the variable for maternity mortality rate is associated with multiple columns. Every row should correspond to one class observation.

In [ ]:
mat_mort1 <- data.frame(country = c("Turkey", "United States", "Sweden", "Japan"),
                       y2002 = c(64, 9.9, 4.17, 7.8),
                       y2007 = c(21.9, 12.7, 1.86, 3.6),
                       y2012 = c(15.2, 16, 5.4, 4.8))
head(mat_mort1)

Instead, we could make this data tidy by having columns for country, year, and maternity mortality rate. Now every observation is associated with an individual row. 

In [16]:
mat_mort2 <- data.frame(country = rep(c("Turkey", "United States", "Sweden", "Japan"), 3),
                       year = c(rep(2002, 4), rep(2007, 4), rep(2012, 4)),
                       mat_mort_rate = c(64.0, 9.9, 4.17, 7.8, 21.9, 12.7, 1.86, 3.6, 15.2, 16, 5.4, 4.8))
head(mat_mort2)

,country,year,mat_mort_rate
,<chr>,<dbl>,<dbl>
1,Turkey,2002,64.00
2,United States,2002,9.90
3,Sweden,2002,4.17
4,Japan,2002,7.80
5,Turkey,2007,21.90
6,United States,2007,12.70


## Reshaping Data

The mobility and covid case data are in tidy form - each observation corresponds to a single row and every column is a single variable. We might consider the lockdown dates to not be in the same form. Another way to represent this data would be to have each observation be the start or end of a stay at home order. 

To reshape our data, we use the `pivot_longer()` function to change the data from what is called **wide form** to **long form**. This kind of pivot involves taking a subset of columns that we will *gather* into a single column while increasing the number of rows in the data set. Before pivoting, we have to think about which columns we are transforming. In our case, we want to take the lockdown start and end columns and create two columns: one column will be whether this is the start or end of a lockdown and the other will be the date. These are called the key and value columns, respectively. The key column will get its values from the names of the columns we are transforming (or the keys) whereas the value column will get its values from the entries in those columns (or the values). 

TODO: image

The `pivot_longer()` function takes in a data table, the columns `cols` that we are pivoting to longer form, the column name `names_to` that will store the data from the previous column names, and the column name `values_to` for the column that will store the information from the columns gathered. In our case, the first column we will name `Lockdown_Event` since it will contain whether each date is the start or end of a lockdown and the second column we will name `Date`. Take a look at the result below.

In [8]:
lockdown_long <- lockdowndates %>%
  pivot_longer(cols=c("Lockdown_Start", "Lockdown_End"), names_to="Lockdown_Event", values_to="Date") %>%
  mutate(Date = as.Date(Date, formula ="%Y-%M-%D"), 
         Lockdown_Event = ifelse(Lockdown_Event=="Lockdown_Start", "Start", "End")) %>%
  na.omit()
head(lockdown_long)

State,Lockdown_Event,Date
<chr>,<chr>,<date>
Alabama,Start,2020-04-04
Alabama,End,2020-04-30
Alaska,Start,2020-03-28
Alaska,End,2020-04-24
Arizona,Start,2020-03-31
Arizona,End,2020-05-15


We can also transform our data in the opposite direction. The function `pivot_wider()` converts data in long form to wide form. This function again takes in the the data frame but now we specify the arguments `names_from` and `values_from`. The former is the column to get the new column names from and the latter is where the row values will be taken from. To pivot our lockdown data back to wider form, we specify that `names_from` is the lockdown event and `values_from` is the date itself. Now we are back to the same form as before!

In [9]:
lockdown_wide <- pivot_wider(lockdown_long, names_from=Lockdown_Event, values_from=Date)
head(lockdown_wide)

State,Start,End
<chr>,<date>,<date>
Alabama,2020-04-04,2020-04-30
Alaska,2020-03-28,2020-04-24
Arizona,2020-03-31,2020-05-15
California,2020-03-19,2020-08-28
Colorado,2020-03-26,2020-04-26
Connecticut,2020-03-23,2020-05-20


Let's show another example. Suppose that I wanted to create a data frame where the columns corresponded to the number of cases for each state in New England and the rows corresponded to the months. First, I need to filter my data to New England and then summarize my data to find the number of cases per month. I use the `month()` function to be able to group by month and state. Additionally, you can see that I add an `ungroup()` at the end since the summarized output will still be grouped by state (as shown in the warning message). 

In [10]:
ne_cases <- covidcases %>% 
  filter(state %in% c("Maine", "Vermont", "New Hampshire", "Connecticut", "Rhode Island",
                                  "Massachusetts")) %>%
  mutate(month = month(date)) %>%
  group_by(state, month) %>%
  summarize(total_cases = sum(weekly_cases)) %>%
  ungroup()
head(ne_cases)

`summarise()` has grouped output by 'state'. You can override using the
`.groups` argument.


state,month,total_cases
<chr>,<dbl>,<int>
Connecticut,3,579
Connecticut,4,20875
Connecticut,5,13211
Connecticut,6,4237
Connecticut,7,2812
Connecticut,8,3388


Now, I need to focus on converting this data to wide format. I want a column for each state. This tells me that my `names_from` argument will be `state`. Next, I want each row to have the case values for each state. This tells me that my `values_from` argument will be total_cases. The format of this data may not be tidy but it allows me to quickly compare cases across states.

In [11]:
pivot_wider(ne_cases, names_from=state, values_from=total_cases)

month,Connecticut,Maine,Massachusetts,New Hampshire,Rhode Island,Vermont
<dbl>,<int>,<int>,<int>,<int>,<int>,<int>
3,579,106,1022,91,NA,80
4,20875,818,49717,1637,6333,817
5,13211,1150,29414,1869,14770,875
6,4237,1304,12289,1319,2088,958
7,2812,1333,7573,847,48876,832
8,3388,884,127787,763,72773,669
9,900,319,2024,136,19809,157


### Practice Question 

Something involving pivots

In [ ]:
# Solution:

## Merging Data with Joins

Above, we saw how to manipulate our current data into a new format. Now, we will see how we can combine our multiple data sources. Merging two data frames is called joining and the function we will use depends on how we want to match between the data frames. The image below shows an overview of the different joins and the video talks through each join type.

**Types of Joins**:
* `left_join(table1,table2,by)`: Joins each row of table1 with all matches in table2 
*  `right_join(table1,table2,by)`: Joins each row of table2 with all matches in table1 (the opposite of a left join) 
* `inner_join(table1,table2,by)`: Looks for all matches between rows in table1 and table2. Rows that do not find a match are dropped.  
* `full_join(table1,table2,by)`: Keeps all rows from both tables and joins those that match. Rows that do not find a match will have NA values filled in.   
* `semi_join(table1,table2,by)`: Keeps all rows in table1 that have a match in table2 but does not join to any information from table2.  
* `anti_join(table1,table2,by)`: Keeps all rows in table1 that *do not* have a match in table 2 but does not join to any information from table2. The opposite of a semi join.  

TODO: image and video

We will first demonstate a left join using the `left_join()` function. This function takes in two data tables (table1 and table 2) and the columns to match rows by. In a left join, for every row of table1, we look for all matching rows in table2 and add any columns not used to do the matching. Thus, every row in table1 corresponds to at least one entry in the resulting table but possibly more if there are multiple matches. We will use a left join to add the lockdown information to our case data. In this case, the first table will be `covidcases` and we will match by `state`. Since the state column is slightly different between the two data frames we specify that `state` is equivalent to `State` in the `by` argument.

In [12]:
covidcases_full <- left_join(covidcases, lockdowndates, by=c("state"="State"))
head(covidcases_full)

state,county,week,weekly_cases,weekly_deaths,date,Lockdown_Start,Lockdown_End
<chr>,<chr>,<dbl>,<int>,<int>,<date>,<date>,<date>
Alabama,Autauga,12,3,0,2020-03-25,2020-04-04,2020-04-30
Alabama,Autauga,13,3,0,2020-04-01,2020-04-04,2020-04-30
Alabama,Autauga,14,2,1,2020-04-08,2020-04-04,2020-04-30
Alabama,Autauga,15,11,1,2020-04-15,2020-04-04,2020-04-30
Alabama,Autauga,16,5,1,2020-04-22,2020-04-04,2020-04-30
Alabama,Autauga,17,8,2,2020-04-29,2020-04-04,2020-04-30


These two new columns will allow us to determine whether a given day was during a lockdown. We use the `between` function to create a new column `lockdown` before dropping the two date columns. We can check that this column worked as expected by choosing a single county to look at. 

In [13]:
covidcases_full <- covidcases_full %>%
  mutate(lockdown = between(date, Lockdown_Start, Lockdown_End)) %>%
  select(-c(Lockdown_Start, Lockdown_End)) 
covidcases_full %>%
  filter(state == "Alabama", county == "Jefferson", date <= as.Date("2020-05-10"))

state,county,week,weekly_cases,weekly_deaths,date,lockdown
<chr>,<chr>,<dbl>,<int>,<int>,<date>,<lgl>
Alabama,Jefferson,11,19,0,2020-03-18,FALSE
Alabama,Jefferson,12,66,0,2020-03-25,FALSE
Alabama,Jefferson,13,153,0,2020-04-01,FALSE
Alabama,Jefferson,14,156,8,2020-04-08,TRUE
Alabama,Jefferson,15,128,2,2020-04-15,TRUE
Alabama,Jefferson,16,72,9,2020-04-22,TRUE
Alabama,Jefferson,17,117,10,2020-04-29,TRUE
Alabama,Jefferson,18,139,5,2020-05-06,FALSE


We now want to add in the mobility data. In the last case, we wanted to keep any observation in `covidcases` regardless if it was in the `lockdowndates` data frame. Therefore, we used a left join. In this case, we will only want to keep observations that have mobility date for that state on each date. This indicates that we want to use an *inner join*. The function `inner_join()` takes in two data tables (table1 and table2) and the columns to match rows by. The function only keeps rows in table1 that match to a row in table2. Again, those columns in table2 not used to match with table1 are added to the resulting outcome. In this case, we match by state and date.

In [14]:
covidcases_full <- inner_join(covidcases_full, mobility, by = c("state", "date")) %>%
  select(-c(samples, m50_index))
head(covidcases_full)

state,county,week,weekly_cases,weekly_deaths,date,lockdown,m50
<chr>,<chr>,<dbl>,<int>,<int>,<date>,<lgl>,<dbl>
Alabama,Autauga,12,3,0,2020-03-25,FALSE,9.693279
Alabama,Autauga,13,3,0,2020-04-01,FALSE,8.525250
Alabama,Autauga,14,2,1,2020-04-08,TRUE,6.983088
Alabama,Autauga,15,11,1,2020-04-15,TRUE,8.585515
Alabama,Autauga,16,5,1,2020-04-22,TRUE,9.964838
Alabama,Autauga,17,8,2,2020-04-29,TRUE,8.776103


### Practice Question

Something about joins

In [ ]:
# Solution:

## Recap Video

TODO: might skip since we have a video up above

## Exercises

1. Why does this code fail?